<a href="https://www.kaggle.com/code/prasaddash/prasad-brats-d1?scriptVersionId=92652238" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import nibabel as nib
import numpy as np 
import matplotlib.pyplot as plt
import os
import random
import torch
import pandas as pd
from tqdm.notebook import tqdm_notebook
from torchvision import transforms, utils
import cv2

### Extracting the files

In [2]:
import tarfile
file = tarfile.open('../input/brats-2021-task1/BraTS2021_Training_Data.tar')

file.extractall('./brain_images')
file.close()

### Visualise the data from one patient for a random layer

In [3]:
# PATIENT_ID='00558'
# T1_PATH=f"./brain_images/BraTS2021_{PATIENT_ID}/BraTS2021_{PATIENT_ID}_t1.nii.gz"
# T1CE_PATH=f"./brain_images/BraTS2021_{PATIENT_ID}/BraTS2021_{PATIENT_ID}_t1ce.nii.gz"
# T2_PATH=f"./brain_images/BraTS2021_{PATIENT_ID}/BraTS2021_{PATIENT_ID}_t2.nii.gz"
# FLAIR_PATH=f"./brain_images/BraTS2021_{PATIENT_ID}/BraTS2021_{PATIENT_ID}_flair.nii.gz"
# MASK_PATH=f"./brain_images/BraTS2021_{PATIENT_ID}/BraTS2021_{PATIENT_ID}_seg.nii.gz"
# img_t1=nib.load(T1_PATH)
# img_t1ce=nib.load(T1CE_PATH)
# img_t2=nib.load(T2_PATH)
# img_flair=nib.load(FLAIR_PATH)
# img_mask=nib.load(MASK_PATH)
# image_data_t1=img_t1.get_fdata()
# image_data_t1ce=img_t1ce.get_fdata()
# image_data_t2=img_t2.get_fdata()
# image_data_flair=img_flair.get_fdata()
# mask=img_mask.get_fdata()
# print(img_t1.header)
# zero_matrix=np.zeros((240,240))
# start=-1
# end=1e9
# nonEmptyMasks=[]
# for i in range(0,155):
#     if not np.array_equal(zero_matrix,mask[:,:,i]):
#         if start==-1:
#                 start=i
#         nonEmptyMasks.append(i)
#         if i<end:
#             end=i

# def one_normal(x):
#     if x==0:
#         return 0
#     else:
#         return 1
# print(image_data_t1.min())
# plt.rcParams.update({'text.color': "red"})
# LAYER=random.choice(nonEmptyMasks)

# # Put the value here and uncomment to see a particular layer
# #LAYER=93

# fig = plt.figure(figsize=(100,50))
# rows = 5
# cols=1
# fig.add_subplot(rows, cols, 1)
# cmask=np.vectorize(one_normal)(mask[:,:,LAYER])
# plt.imshow(image_data_t1[:,:,LAYER]*cmask,cmap='gray')
# plt.axis('off')
# plt.title("T1")
# fig.add_subplot(rows, cols,2)
# plt.imshow(image_data_t1ce[:,:,LAYER]*cmask,cmap='gray')
# plt.axis('off')
# plt.title("T1CE")
# fig.add_subplot(rows, cols, 3)
# plt.imshow(image_data_t2[:,:,LAYER]*cmask,cmap='gray')
# plt.axis('off')
# plt.title("T2")
# fig.add_subplot(rows, cols, 4)
# plt.imshow(image_data_flair[:,:,LAYER]*cmask,cmap='gray')
# plt.axis('off')
# plt.title("FLAIR")
# fig.add_subplot(rows, cols, 5)
# plt.imshow(mask[:,:,LAYER],cmap='gray')
# plt.axis('off')
# plt.title("MASK")
# fig.suptitle(f'Layer Number {LAYER}', fontsize=20)

### To find the global maximums and minimums 
( Not being used now)

In [4]:
# MAX_T1=-1
# MIN_T1=1e9
# for i in tqdm_notebook(ALL_MGMT_PATIENTS,desc='T1'):
#     img_t1=nib.load(f"./brain_images/BraTS2021_{i}/BraTS2021_{i}_t1.nii.gz")
#     image_data=img_t1.get_fdata()
#     MAX_T1=max(MAX_T1,image_data.max())
#     print(i,image_data.min())
#     MIN_T1=min(MIN_T1,image_data.min())
# print("Max Intensity in T1 Images : ",MAX_T1)
# print("Min Intensity in T1 Images : ",MIN_T1)
# MAX_T1CE=-1
# MIN_T1CE=1e9
# for i in tqdm_notebook(ALL_MGMT_PATIENTS,desc='T1CE'):
#     img_t1=nib.load(f"./brain_images/BraTS2021_{i}/BraTS2021_{i}_t1ce.nii.gz")
#     image_data=img_t1.get_fdata()
#     MAX_T1CE=max(MAX_T1CE,image_data.max())
#     MIN_T1CE=min(MIN_T1CE,image_data.min())
# print("Max Intensity in T1 Images : ",MAX_T1CE)
# print("Min Intensity in T1 Images : ",MIN_T1CE)
# MAX_T2=-1
# MIN_T2=1e9
# for i in tqdm_notebook(ALL_MGMT_PATIENTS,desc='T2'):
#     img_t1=nib.load(f"./brain_images/BraTS2021_{i}/BraTS2021_{i}_t2.nii.gz")
#     image_data=img_t1.get_fdata()
#     MAX_T2=max(MAX_T2,image_data.max())
#     MIN_T2=min(MIN_T2,image_data.min())
# print("Max Intensity in T2 Images : ",MAX_T2)
# print("Min Intensity in T2 Images : ",MIN_T2)
# MAX_FLAIR=-1
# MIN_FLAIR=1e9
# for i in tqdm_notebook(ALL_MGMT_PATIENTS,desc='FLAIR'):
#     img_t1=nib.load(f"./brain_images/BraTS2021_{i}/BraTS2021_{i}_flair.nii.gz")
#     image_data=img_t1.get_fdata()
#     MAX_FLAIR=max(MAX_FLAIR,image_data.max())
#     MIN_FLAIR=min(MIN_FLAIR,image_data.min())
# print("Max Intensity in FLAIR Images : ",MAX_FLAIR)
# print("Min Intensity in FLAIR Images : ",MIN_FLAIR)

**The following block of code provides the list of all the slices which have more than 300 nno zero pixels in masks 
The list ALL_DATA has tuples of form (patient_id, slice_no ,label)**

In [5]:
labels=pd.read_csv('../input/mgmt-labels/train_labels_MGMT.csv')
CURRENT_FOLDER='./brain_images'
ALL_PATIENTS=[]
for f in tqdm_notebook(os.listdir(CURRENT_FOLDER),desc='Progress'):
    path_to_dir=os.path.join(CURRENT_FOLDER,f)
    if os.path.isdir(path_to_dir):
        ALL_PATIENTS.append(f)
ALL_MGMT_PATIENTS=[str(label).zfill(5) for label in labels['BraTS21ID']]
print("Total Number of patients : ",len(ALL_PATIENTS))
print("Total Number of patients with MGMT Data : ",len(ALL_MGMT_PATIENTS))
print("Total Number of patients with MGMT Methylation Positive : ",len(labels[labels['MGMT_value']==1]))
print("Total Number of patients with MGMT Methylation Negative : ",len(labels[labels['MGMT_value']==0]))
print("The following patients donot have corresponding MRI Files")
for i in ALL_MGMT_PATIENTS:
    if f'BraTS2021_{i}' not in ALL_PATIENTS:
        print(i)
ALL_MGMT_PATIENTS=[i for i in ALL_MGMT_PATIENTS if f'BraTS2021_{i}' in ALL_PATIENTS]
NEGATIVE_IMAGES=0
POSITIVE_IMAGES=0
ALL_DATA=[]
for i in tqdm_notebook(ALL_MGMT_PATIENTS,desc='Progress'):
    zero_matrix=np.zeros((240,240))
    start=-1
    end=1e9
    nonEmptyMasks=[]
    MASK_PATH=f"./brain_images/BraTS2021_{i}/BraTS2021_{i}_seg.nii.gz"
    img_mask=nib.load(MASK_PATH)
    mask=img_mask.get_fdata()
    for j in range(0,155):
        if (mask[:,:,j]>0).astype('float64').sum()>300 :
            ALL_DATA.append((i,j,labels[labels['BraTS21ID']==int(i)]['MGMT_value'].values[0])) 
            if(labels[labels['BraTS21ID']==int(i)]['MGMT_value'].values[0]):
                POSITIVE_IMAGES+=1
            else:
                NEGATIVE_IMAGES+=1

Progress:   0%|          | 0/1252 [00:00<?, ?it/s]

Total Number of patients :  1251
Total Number of patients with MGMT Data :  585
Total Number of patients with MGMT Methylation Positive :  307
Total Number of patients with MGMT Methylation Negative :  278
The following patients donot have corresponding MRI Files
00169
00197
00245
00308
00408
00564
00794
00998


Progress:   0%|          | 0/577 [00:00<?, ?it/s]

In [6]:
print("Final No. of Images in the dataset:",len(ALL_DATA))
print("No of Image Slices labelled true",POSITIVE_IMAGES)
print("No of Images Slices labelled false",NEGATIVE_IMAGES)

Final No. of Images in the dataset: 30976
No of Image Slices labelled true 16387
No of Images Slices labelled false 14589


# Model Declarations


Using vgg16 and resnet 18

### Downloading pretrained models from torch hub

In [7]:
import torchvision.models as models
import torch.nn as nn
model_vgg=models.vgg16(pretrained=True)
model_resnet=models.resnet18(pretrained=True)

#Take the first two layers i.e. Linear Layer and ReLU layer
model_vgg.classifier = model_vgg.classifier[:2]

#model_vgg.classifier = nn.Identity()
# Setting vgg to nn.Identity() procuces 512*7*7 features
model_resnet.fc = nn.Identity()

#Set the training to false
for param in model_vgg.parameters():
    param.requires_grad = False
for param in model_resnet.parameters():
    param.requires_grad = False

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [8]:
!pip install torchsummary
from torchsummary import summary
summary(model_vgg,(3,240,240))
summary(model_resnet,(3,240,240))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 240, 240]           1,792
              ReLU-2         [-1, 64, 240, 240]               0
            Conv2d-3         [-1, 64, 240, 240]          36,928
              ReLU-4         [-1, 64, 240, 240]               0
         MaxPool2d-5         [-1, 64, 120, 120]               0
            Conv2d-6        [-1, 128, 120, 120]          73,856
              ReLU-7        [-1, 128, 120, 120]               0
            Conv2d-8        [-1, 128, 120, 120]         147,584
              ReLU-9        [-1, 128, 120, 120]               0
        MaxPool2d-10          [-1, 128, 60, 60]               0
           Conv2d-11          [-1, 256, 60, 60]         295,168
             ReLU-12          [-1, 256, 60, 60]               0
           Conv2d-13          [-1, 256, 60, 60]         590,080
             ReLU-14          [-1, 256,

**Extract Region takes in a masked image and extracts the tumor region out of it by finding the first non-zero row/column in all directions. The image is then interpolated to size (240,240) and subsequently converted to RBG .**

In [9]:
def ExtractRegion(img):
    img_x,img_y=img.shape
    for i in range(img_x):
        if img[i,:].sum()>0:
            ul=i
            break
    for i in range(img_x-1,0,-1):
        if img[i,:].sum()>0:
            dl=i
            break
    for i in range(img_y):
        if img[:,i].sum()>0:
            ll=i
            break
    for i in range(img_y-1,0,-1):
        if img[:,i].sum()>0:
            rl=i
            break
    img = img[ul:dl+1,ll:rl+1]
    img =  cv2.resize(img, (240,240), interpolation = cv2.INTER_CUBIC)
    img = cv2.cvtColor(img.astype('float32'),cv2.COLOR_GRAY2RGB)
    #The downloaded models expects input in form of (B,C,H,W) so the image is reshaped 
    img = img.reshape((3,240,240))
    return img

In [10]:
ALL_DATA = ALL_DATA[:5000]
X = [(i,j) for i,j,_ in tqdm_notebook(ALL_DATA,desc='Data')]
y = [ i for _,j,i in tqdm_notebook(ALL_DATA,desc='Labels')]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

Data:   0%|          | 0/5000 [00:00<?, ?it/s]

Labels:   0%|          | 0/5000 [00:00<?, ?it/s]

**individual_feature_extractor**

This function takes in a patient_id , slice_no and returns the features extracted from vgg and resnet.

In [11]:
def individual_feature_extractor(patient_id : str,slice_no : int):
    img = nib.load(f"./brain_images/BraTS2021_{patient_id}/BraTS2021_{patient_id}_t1.nii.gz").get_fdata()
    img = (img-img.min())/(img.max()-img.min())
    mask = nib.load(f"./brain_images/BraTS2021_{patient_id}/BraTS2021_{patient_id}_seg.nii.gz").get_fdata()
    slice = img[:,:,slice_no] * (mask[:,:,slice_no]>0).astype('float')
    slice = ExtractRegion(slice)
    slice = torch.tensor([slice])
    vgg_results = model_vgg(slice).numpy()
    resnet_results = model_resnet(slice).numpy()
    combined_output = np.concatenate((vgg_results,resnet_results),axis = 1)
    combined_output = np.array(combined_output)
    return combined_output

In [12]:
## Creating the trainset
X_train_data =  [individual_feature_extractor(i,j) for i,j in tqdm_notebook(X_train,desc='train-data')]
X_train_data=np.array(X_train_data)
X_train_data=X_train_data.reshape((3750,4608))

train-data:   0%|          | 0/3750 [00:00<?, ?it/s]

### Feature Selection

In [13]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
X_temp = X_train_data.copy()
sel = VarianceThreshold(threshold=(0.01))
X_temp=sel.fit_transform(X_temp)
X_temp.shape

(3750, 1583)

**Creating the test data from selected features**

In [14]:
X_test_data =  [individual_feature_extractor(i,j) for i,j in tqdm_notebook(X_test,desc='test-data')]
X_test_data = np.array(X_test_data)
X_test_data =  X_test_data.reshape((1250,4608)) 
X_test_temp=sel.transform(X_test_data)

test-data:   0%|          | 0/1250 [00:00<?, ?it/s]

# Trying out different Classifiers

## SVM with *rbf* kernel

In [15]:
from sklearn.svm import SVC
clf =  SVC(kernel='rbf')
clf.fit(X_temp,y_train)
preds = clf.predict(X_test_temp)
from sklearn.metrics import classification_report
print(classification_report(y_test,preds))   

              precision    recall  f1-score   support

           0       0.74      0.48      0.58       568
           1       0.66      0.86      0.75       682

    accuracy                           0.68      1250
   macro avg       0.70      0.67      0.66      1250
weighted avg       0.70      0.68      0.67      1250



## KNN classifier

In [16]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors = 5)
neigh.fit(X_temp, y_train)
preds = neigh.predict(X_test_temp)
print(classification_report(y_test,preds))

              precision    recall  f1-score   support

           0       0.89      0.85      0.87       568
           1       0.88      0.91      0.89       682

    accuracy                           0.88      1250
   macro avg       0.88      0.88      0.88      1250
weighted avg       0.88      0.88      0.88      1250



## Decesion tree and Random forest

In [17]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_temp, y_train)
preds = clf.predict(X_test_temp)
print(classification_report(y_test,preds))

              precision    recall  f1-score   support

           0       0.64      0.64      0.64       568
           1       0.70      0.71      0.70       682

    accuracy                           0.68      1250
   macro avg       0.67      0.67      0.67      1250
weighted avg       0.68      0.68      0.68      1250



In [18]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=10, random_state=342)
clf.fit(X_temp, y_train)
preds = clf.predict(X_test_temp)
print(classification_report(y_test,preds))

              precision    recall  f1-score   support

           0       0.83      0.56      0.67       568
           1       0.71      0.91      0.80       682

    accuracy                           0.75      1250
   macro avg       0.77      0.73      0.73      1250
weighted avg       0.77      0.75      0.74      1250



## Naive Bayes

In [19]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
clf.fit(X_temp, y_train)
preds = clf.predict(X_test_temp)
print(classification_report(y_test,preds))

              precision    recall  f1-score   support

           0       0.53      0.39      0.45       568
           1       0.59      0.72      0.64       682

    accuracy                           0.57      1250
   macro avg       0.56      0.55      0.55      1250
weighted avg       0.56      0.57      0.56      1250

